# Mini-Challenge Erdbebenmonitor

Text blalba



Loading packages

In [1]:
import pandas as pd
import geopandas as gpd

Loading Data as a GeoJson Object

In [2]:
df = gpd.read_file('https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-02')

df.describe()

,mag,time,updated,tz,felt,cdi,mmi,tsunami,sig,nst,dmin,rms,gap
count,326.000000,3.260000e+02,3.260000e+02,284.000000,1.0,1.0,2.000000,326.000000,326.000000,225.000000,257.000000,326.00000,264.000000
mean,1.479693,1.388578e+12,1.488382e+12,-392.852113,32.0,3.1,4.385500,0.003067,62.134969,16.448889,0.722739,0.26557,125.933258
std,1.367151,2.614801e+07,5.440828e+10,306.939698,NaN,NaN,0.174655,0.055385,101.200852,13.508539,2.242265,0.35099,67.449180
min,-0.900000,1.388534e+12,1.388584e+12,-720.000000,32.0,3.1,4.262000,0.000000,0.000000,3.000000,0.000901,0.00020,14.000000
25%,0.530000,1.388554e+12,1.457722e+12,-480.000000,32.0,3.1,4.323750,0.000000,4.000000,7.000000,0.012610,0.04000,78.250000
50%,1.115000,1.388578e+12,1.486063e+12,-480.000000,32.0,3.1,4.385500,0.000000,19.000000,12.000000,0.056760,0.11000,114.000000
75%,1.900000,1.388602e+12,1.530313e+12,-480.000000,32.0,3.1,4.447250,0.000000,56.000000,20.000000,0.168300,0.39750,156.260000
max,6.500000,1.388620e+12,1.598476e+12,660.000000,32.0,3.1,4.509000,1.000000,650.000000,77.000000,17.795000,3.09000,316.800000


In [3]:
df.columns

Index(['id', 'mag', 'place', 'time', 'updated', 'tz', 'url', 'detail', 'felt',
       'cdi', 'mmi', 'alert', 'status', 'tsunami', 'sig', 'net', 'code', 'ids',
       'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'type',
       'title', 'geometry'],
      dtype='object')

### Attribute description

https://earthquake.usgs.gov/data/comcat/data-eventterms.php

* mag - The magnitude for the event
* place - Textual description of named geographic region near to the event
* time - Time when the event occurred. Times are reported in milliseconds since the epoch ( 1970-01-01T00:00:00.000Z), and do not include leap seconds. In certain output formats, the date is formatted for readability. 
* updated - Time when the event was most recently updated
* tz - Timezone offset from UTC in minutes at the event epicenter
* url - Link to USGS Event Page for event. 
* detail - Link to GeoJSON detail feed from a GeoJSON summary feed. 
* felt - The total number of felt reports submitted to the DYFI? system. 
* cdi - The maximum reported intensity for the event. Computed by DYFI. While typically reported as a roman numeral, for the purposes of this API, intensity is expected as the decimal equivalent of the roman numeral.
* mmi - The maximum estimated instrumental intensity for the event
* alert - The alert level from the PAGER earthquake impact scale. 
* status - Indicates whether the event has been reviewed by a human
* tsunami - This flag is set to "1" for large events in oceanic regions and "0" otherwise
* sig - A number describing how significant the event is. Larger numbers indicate a more significant event. This value is determined on a number of factors, including: magnitude, maximum MMI, felt reports, and estimated impact. 
* net - The ID of a data contributor.
* code - An identifying code assigned by - and unique from - the corresponding source for the event. 
* ids - A comma-separated list of event ids that are associated to an event. 
* sources - A comma-separated list of network contributors. 
* types - A comma-separated list of product types associated to this event. 
* nst - The total number of seismic stations used to determine earthquake location
* dmin - Horizontal distance from the epicenter to the nearest station (in degrees). 1 degree is approximately 111.2 kilometers. In general, the smaller this number, the more reliable is the calculated depth of the earthquake. 
* rms - The root-mean-square (RMS) travel time residual, in sec, using all weights. This parameter provides a measure of the fit of the observed arrival times to the predicted arrival times for this location. Smaller numbers reflect a better fit of the data. The value is dependent on the accuracy of the velocity model used to compute the earthquake location, the quality weights assigned to the arrival time data, and the procedure used to locate the earthquake. 
* gap - The largest azimuthal gap between azimuthally adjacent stations (in degrees). In general, the smaller this number, the more reliable is the calculated horizontal position of the earthquake. Earthquake locations in which the azimuthal gap exceeds 180 degrees typically have large location and depth uncertainties.
* magtype - The method or algorithm used to calculate the preferred magnitude for the event. 
* type - Type of seismic event. 
* title - 
* geometry


### Aufbau

Die Struktur des GeoJson ist hier beschrieben: https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php
Durch den Import mittels Geopandas wird die Struktur des Objekts bereits "flachgedrückt". Alternativ wäre die Möglichkeit Json_Normalize (from pandas import json_normalize) zu benutzen

In [43]:
df.iloc[0].geometry.z

11.008

Unnötige Spalten werden entfernt

In [6]:
df = df.drop(['felt','status','net','code','ids','sources','types','nst','rms','gap','magType'], axis=1)

In [8]:
df.iloc[0]

id                                                 ci11408890
mag                                                      1.29
place                               10km SSW of Idyllwild, CA
time                                            1388620296020
updated                                         1457728844428
tz                                                       -480
url         https://earthquake.usgs.gov/earthquakes/eventp...
detail      https://earthquake.usgs.gov/fdsnws/event/1/que...
cdi                                                       NaN
mmi                                                       NaN
alert                                                    None
tsunami                                                     0
sig                                                        26
dmin                                                  0.06729
type                                               earthquake
title                       M 1.3 - 10km SSW of Idyllwild, CA
geometry

Converting date value to human readable format

In [32]:
df['date'] = pd.to_datetime(df['time'],unit='ms')

In [34]:
df.iloc[0].date

Timestamp('2014-01-01 23:51:36.020000')

Konvertieren der Lon / Lat / Depth Spalten

In [44]:
df['lon'] = df.geometry.x
df['lat'] = df.geometry.y
df['depth'] = df.geometry.z

In [46]:
df.iloc[1]

id                                                ak01421ig3u
mag                                                       1.1
place                           117km NW of Talkeetna, Alaska
time                                            1388620046501
updated                                         1558392330681
tz                                                       -540
url         https://earthquake.usgs.gov/earthquakes/eventp...
detail      https://earthquake.usgs.gov/fdsnws/event/1/que...
cdi                                                       NaN
mmi                                                       NaN
alert                                                    None
tsunami                                                     0
sig                                                        19
dmin                                                      NaN
type                                               earthquake
title                   M 1.1 - 117km NW of Talkeetna, Alaska
geometry